#Intergenerational Advice and Matching: An Experimental Study

## PI: Andrew Schotter and Tingting Ding
---

## The purpose of this program is to parse and transform the existing experiment data files for analysis

### Program Author: Anwar A. Ruff

In [2]:
# Prologue
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame, Series, ExcelFile, ExcelWriter
import datetime

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import os

### Construct a list of all the chat file and their location

In [3]:
# Get the session file names
excel_files = sorted(glob.glob('./ChatWithTypeAndEncoding/*.xls'), reverse=True)

# Session timestamp -> path
file_paths = {os.path.splitext(os.path.basename(path_to_file))[0]:path_to_file for path_to_file in excel_files}

### Read in the session list table, which containing the session IDs and their corresponding timestamp
Note: The session chat data is broken into several files. Each file is named using the timestamp (date-time encoding) during which the session occured.

In [4]:
# Read in the timestamp to session mappings
smap = pd.read_csv('./SessionNum.csv')
smap

,Date,Session Number
0,11092011,1
1,11162011,2
2,11172011,3
3,11222011,4
4,11302011_1045,8
5,11302011_1253,7
6,11302011_1450,6
7,11302011_1645,5
8,12152011,9
9,12202011,10


In [5]:
# Prevent leading zeros from being removed by converting the type to string
smap['Date'] = smap['Date'].astype(str)

### Test: Confirm that we have all of the files specified in the session list

In [6]:
# timestamp
timestamps = smap['Date'].tolist()

# Test: Make sure the dates match the files names
session_missing_files = []
for ts in timestamps:
    if ts not in file_paths:
        session_missing_files.append(ts)

if len(session_missing_files) > 0 :
    print("Error: The following session files are missing:")
    print(session_missing_files)

session_missing_id = []
for ts, path in file_paths.items():
    if ts not in timestamps:
        session_missing_id.append(ts)


if len(session_missing_id) > 0 :
    print("Error: The following session are missing ids:")
    print(session_missing_id)    

###Combine the session metadata into a list indexed by session id - 1

In [7]:
# session id
sids = smap['Session Number'].tolist()

# combine session metada
smetadata = {sids[i]:{"ts": ts, "path":file_paths[ts]} for i, ts in enumerate(timestamps)}

### Load all chat data files and append them into one table

In [8]:
chat = DataFrame(columns=['chatid', 'groupid', 'mechanism', 'homo, hetero or iso', 'mark', 'mark.1', 'conflict', 'final'])
file_rowdata = {}
for session_id, file_metadata in smetadata.items():
    # read chat file
    chatdf = pd.read_excel(file_metadata["path"])
    
    # set the session ID
    chatdf['sid'] = session_id
    
    # convert the columns to lower case
    chatdf.columns = map(str.lower, chatdf.columns)
    
    # Ignore any orphaned data used as a scratch pad below the primary data
    chatdf = chatdf[pd.notnull(chatdf['chatid'])]
    
    # Ignore duplicate columns (i.e. instances where column labels were repeated)
    chatdf = chatdf[chatdf['chatid'].map(np.isreal)]
    
    # chat file row data
    file_rowdata[session_id] = chatdf.shape[0]
    
    chat = chat.append(chatdf)

### Test: row_info is used to look at the overall number of observations. Outliers were checked to make sure they were not a result of erroneous or orphaned data.

In [9]:
row_info = DataFrame(list(file_rowdata.items()), columns=['Session', 'Rows'])
row_info.sort('Session')

,Session,Rows
0,1,200
1,2,44
2,3,73
3,4,124
4,5,180
5,6,191
6,7,139
7,8,146
8,9,110
9,10,103


In [10]:
# Remove NaN from indicator columns
chat[['mark', 'mark.1', 'conflict', 'final']] = chat[['mark','mark.1', 'conflict', 'final']].fillna('')

### Rename columns

In [11]:
chat = chat.rename(columns={
    'chatid':'subject', 'final':'resolution', 'mark':'first-eval', 'mark.1':'second-eval', 'homo, hetero or iso':'hom-het-iso','groupid':'group', 'sid':'session'})
chat.head()

,subject,conflict,resolution,group,hom-het-iso,first-eval,second-eval,mechanism,network,session,text,time,type
0,5,,,1,2,,,1,1,1,hello,285,5
1,2,,,1,2,,,1,1,1,"hi, this is subject2",278,2
2,3,,,1,2,,,1,1,1,hey,275,3
3,4,,,1,2,,,1,1,1,subject 4!,268,4
4,1,,,1,2,,,1,1,1,isubject 1,262,1


### Evaluate indicator encodings and create a column for each indicator. 

### Messages have been separately classified by two separate parties (first-eval, second-eval), and a third party has resolved any conflicts (resolution). The identifier coluns are produced by inspecting the resolution column. If a label is in the resolution column, or it is one of the evaluation columns but not in a conflict column, a 1 is placed in the corresponding identifier column for the row message.

In [12]:
indicator_evaluation_table = chat[['resolution', 'first-eval', 'second-eval', 'conflict']]

In [13]:
indicator_labels = ['t-', 't', 'p-', 'p', 'i-', 'i', 's', 'l', 'c'];

for label in indicator_labels:
    # Set rows to true if it contains the indicator string. The remaining rows are NaN so those are set to false.
    in_resolution = indicator_evaluation_table['resolution'].str.contains(label.upper()).fillna(False)

    # the same process is employed above is used for the two evaluation columns as well as the conflict column.
    in_eone = indicator_evaluation_table['first-eval'].str.contains(label.upper()).fillna(False)
    in_etwo = indicator_evaluation_table['second-eval'].str.contains(label.upper()).fillna(False)
    in_conflict = indicator_evaluation_table['conflict'].str.contains(label.upper()).fillna(False)

    # Remove the read indicator from the eval table. 
    # Note: This is due to the choice of indicators wich can result in false positives
    #       For example 'T' is in 'T-'. To avoid this negative indicators are parsed, before
    #       their positive couterparts, and removed.
    indicator_table = indicator_evaluation_table.replace({'.*{}.*'.format(label.upper()):'',}, regex=True)

    # If the current indicator is found in the resolution column we record the observation.
    # We also record the observation of the indicator is in either the evaluation columns (eone, etwo),
    # and not in the conflict column (conflict).
    encoding_col = in_resolution | ((in_eone | in_etwo) & ~in_conflict)
    chat[label] = encoding_col.astype(int)

chat.head()

,subject,conflict,resolution,group,hom-het-iso,first-eval,second-eval,mechanism,network,session,...,type,t-,t,p-,p,i-,i,s,l,c
0,5,,,1,2,,,1,1,1,...,5,0,0,0,0,0,0,0,0,0
1,2,,,1,2,,,1,1,1,...,2,0,0,0,0,0,0,0,0,0
2,3,,,1,2,,,1,1,1,...,3,0,0,0,0,0,0,0,0,0
3,4,,,1,2,,,1,1,1,...,4,0,0,0,0,0,0,0,0,0
4,1,,,1,2,,,1,1,1,...,1,0,0,0,0,0,0,0,0,0


### Retrieve non-chat session data

In [14]:
nonchat = pd.read_excel('./NoChat/AllSessionDataWithTypeStrategyMechanismNetwork.xlsx')
nonchat.head()

,obv,sid,chatid,type,sp1,sp2,mech,net
0,1,1,9,3,1,2,1,1
1,2,1,6,1,2,2,1,1
2,3,1,11,4,2,2,1,1
3,4,1,1,1,2,1,1,1
4,5,1,17,2,2,2,1,1


### Rename non-chat column names

In [15]:
nonchat = nonchat.rename(columns={
    'chatid':'subject', 'sp1':'preference-1', 'sp2':'preference-2','obv':'observation', 'sid':'session'})
nonchat = nonchat.drop(['mech', 'net', 'type'], 1)
nonchat.head()

,observation,session,subject,preference-1,preference-2
0,1,1,9,1,2
1,2,1,6,2,2
2,3,1,11,2,2
3,4,1,1,2,1
4,5,1,17,2,2


### Merge chat and non-chat data into one master observations table

In [16]:
observations = pd.merge(chat, nonchat, on=['session', 'subject'])
# Set dataframe display width
pd.set_option('display.max_columns', 30)
observations.head()

,subject,conflict,resolution,group,hom-het-iso,first-eval,second-eval,mechanism,network,session,text,time,type,t-,t,p-,p,i-,i,s,l,c,observation,preference-1,preference-2
0,5,,,1,2,,,1,1,1,hello,285,5,0,0,0,0,0,0,0,0,0,11,1,1
1,5,,,1,2,,,1,1,1,what type is everyone?,254,5,0,0,0,0,0,0,0,0,0,11,1,1
2,5,,,1,2,,,1,1,1,im type 2,246,5,0,0,0,0,0,0,0,0,0,11,1,1
3,5,PT,P T,1,2,P,T,1,1,1,im going for A since i have priotity and it gi...,197,5,0,1,0,1,0,0,0,0,0,11,1,1
4,5,,,1,2,,,1,1,1,well,191,5,0,0,0,0,0,0,0,0,0,11,1,1


### Write observation table to an excel file (observations.xlsx)

In [17]:
writer = ExcelWriter('./observations.xlsx')
observations.to_excel(writer, 'Observations', na_rep='NA')
writer.save()

### Serialize (pickle) the observations dataframe 

In [18]:
observations.to_pickle('./observations.p')